In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName("NYC_Taxi_Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/04 20:37:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.parquet("NYC/*.parquet")

# Start spark session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYC_Taxi_Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/04 21:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/04 21:36:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Load Data

In [3]:
cwd = os.getcwd()

# For convenience, I save the data in the NYC folder which is in the same directory as this notebook
parquet_file_path = os.path.join(cwd, "NYC", "*.parquet")
taxi_df = spark.read.parquet(parquet_file_path)

# Trip Analysis

## Calculate Duration and Distance:
Create a new column for trip duration and calculate it using the difference between pickup and dropoff times. Also, calculate the average distance for each record:

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

taxi_df = taxi_df.withColumn("trip_duration", F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime"))

In [5]:
taxi_df = taxi_df.withColumn("avg_distance", (F.col("trip_distance") / F.col("passenger_count")))

## Extract Time of Day, Day of Week, and Month of Year:
- Extract the desired time components from the tpep_pickup_datetime column

In [6]:
taxi_df = taxi_df.withColumn("pickup_hour", F.hour("tpep_pickup_datetime"))
taxi_df = taxi_df.withColumn("pickup_day_of_week", F.dayofweek("tpep_pickup_datetime"))
taxi_df = taxi_df.withColumn("pickup_month", F.month("tpep_pickup_datetime"))

## Group and Aggregate:
Group the data by time of day, day of week, and month of year, and calculate the average duration and distance for each group:

In [7]:
agg_df = taxi_df.groupBy("pickup_hour", "pickup_day_of_week", "pickup_month").agg(
    F.avg("trip_duration").alias("avg_duration"),
    F.avg("avg_distance").alias("avg_distance")
).orderBy("pickup_hour", "pickup_day_of_week", "pickup_month")

## Show the Results:
Show the aggregated results:

In [8]:
agg_df.show()

+-----------+------------------+------------+------------------+------------------+
|pickup_hour|pickup_day_of_week|pickup_month|      avg_duration|      avg_distance|
+-----------+------------------+------------+------------------+------------------+
|          0|                 1|           1| 888.9417608770127|3.2521391261294736|
|          0|                 1|           2| 726.9726522187823|  2.89541996285979|
|          0|                 1|           3| 888.4640534063677|2.8471578638497625|
|          0|                 1|           4| 854.6067429406037|2.9782131248083417|
|          0|                 1|           5| 883.9497659700705|2.8323654661521616|
|          0|                 1|           6| 954.6639178045153| 2.721734596752614|
|          0|                 1|           7|1011.6342042755344| 2.835302943232113|
|          0|                 1|           8| 997.7019489609131|2.8669667749237213|
|          0|                 1|           9|1057.5752172184677|2.5259332031

## Group and Count Pickup Locations:

In [9]:
pickup_locations = taxi_df.groupBy("PULocationID").count().orderBy(F.desc("count"))
top_pickup_locations = pickup_locations.limit(10)

In [10]:
dropoff_locations = taxi_df.groupBy("DOLocationID").count().orderBy(F.desc("count"))
top_dropoff_locations = dropoff_locations.limit(10)

In [12]:
print("Top 10 Pickup Locations:")
pickup_locations.show()

Top 10 Pickup Locations:
+------------+-------+
|PULocationID|  count|
+------------+-------+
|         237|1553554|
|         236|1424614|
|         161|1091329|
|         132|1025063|
|         186|1019650|
|         142| 989927|
|         170| 967766|
|         162| 954917|
|         239| 932473|
|         141| 909845|
|          48| 880679|
|         234| 824127|
|         163| 798825|
|          79| 775824|
|         107| 752810|
|         238| 742175|
|         263| 736487|
|          68| 724723|
|         230| 721741|
|         140| 701603|
+------------+-------+
only showing top 20 rows



In [13]:
print("Top 10 Dropoff Locations:")
dropoff_locations.show()

Top 10 Dropoff Locations:
+------------+-------+
|DOLocationID|  count|
+------------+-------+
|         236|1434919|
|         237|1356518|
|         161|1001077|
|         170| 920433|
|         141| 902052|
|         239| 886837|
|         142| 854324|
|          48| 782803|
|         238| 779046|
|         162| 772823|
|         234| 719605|
|         263| 700601|
|          68| 692253|
|         140| 688235|
|         163| 685523|
|         229| 677775|
|         186| 662011|
|         230| 649051|
|          79| 638139|
|         107| 612836|
+------------+-------+
only showing top 20 rows



# Tip analysis

- Calculate the tip percentage for each trip. This involves dividing the tip amount by the total fare amount

In [14]:
tip_analysis_df = taxi_df.withColumn("tip_percentage", F.col("tip_amount") / F.col("total_amount") * 100)
tip_analysis_df = tip_analysis_df.withColumn("tip_percentage", F.when(F.col("tip_percentage") <= 100, F.col("tip_percentage")).otherwise(0))  # Handle possible outliers

# Group by pickup and dropoff locations and calculate average tip percentage and average distance
tip_location_analysis = tip_analysis_df.groupBy("PULocationID", "DOLocationID").agg(
    F.avg("tip_percentage").alias("avg_tip_percentage"),
    F.avg("trip_distance").alias("avg_distance")
).orderBy(F.desc("avg_tip_percentage"))

- Calculate the average tip percentage and total tip amount for each time interval

In [15]:
tip_time_analysis = tip_analysis_df.groupBy("pickup_hour", "pickup_day_of_week", "pickup_month").agg(
    F.avg("tip_percentage").alias("avg_tip_percentage"),
    F.sum("tip_amount").alias("total_tip_amount")
).orderBy("pickup_hour", "pickup_day_of_week", "pickup_month")


In [16]:
print("Tip Analysis by Location:")
tip_location_analysis.show()

Tip Analysis by Location:


+------------+------------+------------------+-----------------+
|PULocationID|DOLocationID|avg_tip_percentage|     avg_distance|
+------------+------------+------------------+-----------------+
|         187|         251|56.179775280898866|             1.54|
|         176|         176|          53.90625|             0.32|
|          96|         236| 48.85197850512946|            11.31|
|         109|         172|46.948356807511736|             2.09|
|         251|         161|46.200737170399776|            19.58|
|         120|         151| 43.01075268817204|             0.73|
|         118|         214|  41.3564929693962|             4.16|
|         172|         214| 39.96670910603205|3.293333333333333|
|         208|         114| 38.55192080359299|            8.975|
|          34|           1| 38.41764929631039|            15.11|
|          82|         253| 38.28483920367534|             2.48|
|         112|         214| 37.67972235994051|            16.03|
|          96|         17

In [17]:
print("Tip Analysis by Time:")
tip_time_analysis.show()

Tip Analysis by Time:


+-----------+------------------+------------+------------------+------------------+
|pickup_hour|pickup_day_of_week|pickup_month|avg_tip_percentage|  total_tip_amount|
+-----------+------------------+------------+------------------+------------------+
|          0|                 1|           1|10.509465545793379| 6285.209999999994|
|          0|                 1|           2|10.960206819588734|  8032.64000000002|
|          0|                 1|           3|11.105551868685199|12657.859999999993|
|          0|                 1|           4|11.314341113424645|18623.729999999974|
|          0|                 1|           5|12.117369325323516|36288.720000000074|
|          0|                 1|           6| 12.19923946046261| 36665.33000000003|
|          0|                 1|           7|11.893981566402744| 34842.08999999987|
|          0|                 1|           8|12.147081300758224|47726.230000000294|
|          0|                 1|           9|12.178405026405924|43128.000000

- Group the data by payment type and calculate statistics such as the average tip percentage, average tip amount, and total tip amount for each payment type

In [18]:
payment_tip_analysis = tip_analysis_df.groupBy("payment_type").agg(
    F.avg("tip_percentage").alias("avg_tip_percentage"),
    F.avg("tip_amount").alias("avg_tip_amount"),
    F.sum("tip_amount").alias("total_tip_amount")
)

In [19]:
print("Payment and Tip Analysis:")
payment_tip_analysis.show()

Payment and Tip Analysis:


+------------+--------------------+--------------------+-------------------+
|payment_type|  avg_tip_percentage|      avg_tip_amount|   total_tip_amount|
+------------+--------------------+--------------------+-------------------+
|           0|   8.070807011379319|  2.1700068168216093| 3208778.2300000293|
|           5|                 0.0|                 0.0|                0.0|
|           1|  15.331362787552713|  3.0755510306721945|6.913956203000465E7|
|           3|   0.145542231109957|-0.01167058060330...|-1799.0200000000004|
|           2|-0.00155152365292...|4.108590704647675E-4| 2740.4299999999994|
|           4| -0.3495420614922692|0.022958282745690756|            2779.72|
+------------+--------------------+--------------------+-------------------+



# Fare Analysis:

- Calculate the average fare by pickup and dropoff location

In [21]:
fare_location_analysis = taxi_df.groupBy("PULocationID", "DOLocationID").agg(
    F.avg("fare_amount").alias("avg_fare")
).orderBy(F.desc("avg_fare"))

- Calculate the average fare for different passenger counts to analyze the correlation between passenger count and fare amount

In [22]:
fare_passenger_analysis = taxi_df.groupBy("passenger_count").agg(
    F.avg("fare_amount").alias("avg_fare")
).orderBy("passenger_count")

- Calculate the Pearson correlation coefficient to investigate the correlation between fare amount and trip distance

In [23]:
fare_distance_correlation = taxi_df.select(F.corr("fare_amount", "trip_distance").alias("correlation")).collect()[0]["correlation"]

In [24]:
print("Average Fare by Pickup & Drop Location:")
fare_location_analysis.show()

Average Fare by Pickup & Drop Location:


+------------+------------+------------------+
|PULocationID|DOLocationID|          avg_fare|
+------------+------------+------------------+
|         154|          28|            1164.0|
|         234|         189| 843.4665424430641|
|           1|         247|             420.0|
|          83|         136|             378.5|
|           5|          74|             306.0|
|          54|         265|             275.5|
|          29|         264|213.75227272727273|
|           2|         265|            200.25|
|           6|         265|            192.25|
|         123|         265|            177.35|
|         235|         115|             170.0|
|         221|         265|             160.0|
|         253|         208|             160.0|
|         112|         109|             155.0|
|         204|         265|           152.375|
|          44|         138|             151.5|
|         118|         265|            151.25|
|          55|           1|             150.0|
|          10

In [25]:
print("Average Fare by Passenger Count:")
fare_passenger_analysis.show()

Average Fare by Passenger Count:
+---------------+------------------+
|passenger_count|          avg_fare|
+---------------+------------------+
|           null| 25.50271663190539|
|            0.0|12.251760226150822|
|            1.0|12.709557736571188|
|            2.0| 13.77639929394148|
|            3.0|13.555663818461737|
|            4.0|14.284687986716264|
|            5.0|12.666400646383593|
|            6.0| 12.75109443706296|
|            7.0| 52.91679487179488|
|            8.0| 49.14408163265307|
|            9.0|             61.35|
|           96.0|              11.5|
|          112.0|               9.0|
+---------------+------------------+



In [26]:
print("Correlation between Fare Amount and Trip Distance:")
print("Correlation coefficient:", fare_distance_correlation)

Correlation between Fare Amount and Trip Distance:
Correlation coefficient: 0.0008730862657094112


# Traffic Analysis

- Calculate the speed per hour

In [27]:
trip_speed_df = taxi_df.withColumn("trip_speed", (F.col("trip_distance") / (F.col("trip_duration") / 3600)))

- Group by Trip Time Intervals:

In [28]:
trip_time_speed_analysis = trip_speed_df.groupBy("PULocationID", "DOLocationID", "pickup_hour", "pickup_day_of_week", "pickup_month").agg(
    F.avg("trip_speed").alias("avg_trip_speed")
).orderBy("PULocationID", "DOLocationID", "pickup_hour", "pickup_day_of_week", "pickup_month")

In [29]:
trip_time_speed_analysis.show()

23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/04 21:47:53 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+------------+-----------+------------------+------------+-----------------+
|PULocationID|DOLocationID|pickup_hour|pickup_day_of_week|pickup_month|   avg_trip_speed|
+------------+------------+-----------+------------------+------------+-----------------+
|           1|           1|          0|                 1|           4|              0.0|
|           1|           1|          0|                 2|           3|              0.0|
|           1|           1|          0|                 2|          11|              0.0|
|           1|           1|          0|                 4|           6|              0.0|
|           1|           1|          0|                 7|           5|2438.181818181818|
|           1|           1|          1|                 1|           6|              0.0|
|           1|           1|          1|                 1|          11|              0.0|
|           1|           1|          1|                 2|           3|              0.0|
|         

# Demand Analysis

- Create features from the date and time of pickups

In [30]:
from pyspark.ml.feature import VectorAssembler

demand_df = taxi_df.withColumn("pickup_hour", F.hour("tpep_pickup_datetime"))
demand_df = demand_df.withColumn("pickup_day_of_week", F.dayofweek("tpep_pickup_datetime"))
demand_df = demand_df.withColumn("pickup_month", F.month("tpep_pickup_datetime"))

feature_columns = ["pickup_hour", "pickup_day_of_week", "pickup_month"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
demand_df = assembler.transform(demand_df)

- Regression Model (Linear Regression):

In [41]:
feature_columns = ["pickup_hour", "pickup_day_of_week", "pickup_month"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="input_features")
demand_df = assembler.transform(demand_df)

In [42]:
# Aggregate data for regression
regression_df = demand_df.groupBy("pickup_hour").agg(
    F.sum("passenger_count").alias("total_pickups"),
    F.first("input_features").alias("features")  # Take any value since they're the same for each group
)

In [43]:
from pyspark.ml.regression import LinearRegression


lr = LinearRegression(featuresCol="features", labelCol="total_pickups")
lr_model = lr.fit(regression_df)

23/08/04 22:01:25 WARN Instrumentation: [b9fdd9b6] regParam is zero, which might cause numerical instability and overfitting.
23/08/04 22:01:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/08/04 22:01:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/08/04 22:01:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


- Making prediction

In [62]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.sql.functions import hour, dayofweek, month, col
from datetime import timedelta

# Get the last record in the Parquet data
last_record = taxi_df.orderBy(F.desc("tpep_pickup_datetime")).limit(1).collect()[0]

# Extract relevant information from the last record
last_pickup_datetime = last_record["tpep_pickup_datetime"]
next_pickup_datetime = last_pickup_datetime + timedelta(hours=1)

# Extract features for the next hour
next_hour = next_pickup_datetime.hour
next_day_of_week = next_pickup_datetime.weekday()
next_month = next_pickup_datetime.month

# Create a DataFrame for the next hour
next_pickup_row_hour = Row(pickup_hour=next_hour, pickup_day_of_week=next_day_of_week, pickup_month=next_month)
next_pickup_df_hour = spark.createDataFrame([next_pickup_row_hour])

# Convert columns to feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=["pickup_hour", "pickup_day_of_week", "pickup_month"],
                            outputCol="features")

# Transform the features and make predictions for the next hour
next_pickup_df_hour = assembler.transform(next_pickup_df_hour)
predictions_hour = lr_model.transform(next_pickup_df_hour)
predicted_pickups_hour = predictions_hour.select("prediction").collect()[0]["prediction"]

print("Predicted Pickups for Next Hour:", predicted_pickups_hour)


Predicted Pickups for Next Hour: 1341885.7375189392


In [71]:
# Calculate the next day and month
next_day_datetime = last_pickup_datetime + timedelta(days=1)
next_day_hour = next_day_datetime.hour
next_day_of_week = next_day_datetime.weekday()
next_month = next_day_datetime.month

# Create a DataFrame for the next day
next_pickup_row_day = Row(pickup_hour=next_day_hour, pickup_day_of_week=next_day_of_week, pickup_month=next_month)
next_pickup_df_day = spark.createDataFrame([next_pickup_row_day])

# Convert columns to feature vector using VectorAssembler
next_pickup_df_day = assembler.transform(next_pickup_df_day)

# Make predictions for the next day
predictions_day = lr_model.transform(next_pickup_df_day)
predicted_pickups_day = predictions_day.select("prediction").collect()[0]["prediction"]

print("Predicted Pickups for Next Day:", predicted_pickups_day)


Predicted Pickups for Next Day: 1461753.3968590284


In [72]:
# Calculate the next month
next_month_datetime = last_pickup_datetime + timedelta(days=30)  # Assuming 30 days in a month
next_month_hour = next_month_datetime.hour
next_day_of_week = next_month_datetime.weekday()
next_month = next_month_datetime.month

# Create a DataFrame for the next month
next_pickup_row_month = Row(pickup_hour=next_month_hour, pickup_day_of_week=next_day_of_week, pickup_month=next_month)
next_pickup_df_month = spark.createDataFrame([next_pickup_row_month])

# Convert columns to feature vector using VectorAssembler
next_pickup_df_month = assembler.transform(next_pickup_df_month)

# Make predictions for the next month
predictions_month = lr_model.transform(next_pickup_df_month)
predicted_pickups_month = predictions_month.select("prediction").collect()[0]["prediction"]

print("Predicted Pickups for Next Month:", predicted_pickups_month)


Predicted Pickups for Next Month: 1198863.0639412608
